# Generating the NetCDF Grid file for MOM6+CESM


This notebook describes how to generate the **netcdf grid file** for MOM6. This grid file can then be used to generate SCRIP file (for ESMF mapping tools), binary grid files (for CICE), mapping files (for CESM). See ```gen_cesm_files.ipynb``` notebook file for generating those files. First, to generate the netcdf grid file, run the Python script presented in this notebook.

## 0. Prerequisities

First, load the following modules:
 - Python
 - numpy
 - netCDF4
 - matplotlib
 - pyngl
 
On cheyenne, for example:

```
module load python/2.7.13
module load numpy
module load netcdf4-python/1.2.7
module load matplotlib
module load pyngl
```

## 1. Generate the netcdf grid file

MOM6 itself reads in a supergrid, a refined version of the actual horizontal grid, to generate its own data structures. This gives the flexibility of aggregating the supergrid into the B, T, and U grids (?). CESM and CIME, however, requires the actual model (ocn) grid for coupling. 

We first generate a netcdf file containing grid variables, e.g., coordinates, areas, masks, etc. with the normal resolution (coarser than super grid). This file will be used to generate the SCRIP file. The following Python script generates this netcdf file. NOTE: Make sure to update the input file directories and names if necessary.

Import all the necessary Python modules:

In [4]:
import numpy as np
import netCDF4
import numpy as np
from mpl_toolkits.basemap import pyproj
from datetime import datetime
import Ngl

Read in the MOM6 supergrid, which is usually four times the resolution of the actual grid (4n).

In [5]:
# MOM6 supergrid:
nc_sgrd = netCDF4.Dataset('/glade/p/work/gmarques/cesm/grid/MOM6/tx0.66v1/tx0.66v1_hgrid_2018-03-27.nc')
x = nc_sgrd.variables['x'][:]
y = nc_sgrd.variables['y'][:]
dx = nc_sgrd.variables['dx'][:]
dy = nc_sgrd.variables['dy'][:]
area = nc_sgrd.variables['area'][:]
angle_dx = nc_sgrd.variables['angle_dx'][:]
nc_sgrd.close()

Read in the mask and the topography file which should have the normal resolution (n). 

In [3]:
# Mask:
nc_msk = netCDF4.Dataset('/glade/p/work/gmarques/cesm/grid/MOM6/tx0.66v1/tx0.66v1_ocean_topo_edited_180405.nc')
tmask = nc_msk.variables['mask'][:]
nc_msk.close()

# Topography
nc_topo = netCDF4.Dataset('/glade/p/work/gmarques/cesm/grid/MOM6/tx0.66v1/tx0.66v1_ocean_topo_edited_180405.nc')
depth = nc_topo.variables['depth'][:]
nc_topo.close()

By copying every 2nd element, reduce the resolution of the supergrid fields from (4n) to (n).

In [6]:
# T point locations
tlon = x[1::2,1::2]
tlat = y[1::2,1::2]

# U point locations
ulon = x[1::2,::2]
ulat = y[1::2,::2]

# V point locations
vlon = x[::2,1::2]
vlat = y[::2,1::2]

# Corner point locations
qlon = x[::2,::2]
qlat = y[::2,::2]

# T cell area (sum of four supergrid cells)
tarea = area[::2,::2] + area[1::2,1::2] + area[::2,1::2] + area[::2,1::2]

# t-point angle
angle = angle_dx[1::2,1::2]

# x-distance between u-points, centered at t
dxt = dx[1::2,::2] + dx[1::2,1::2]

# y-distance between v-points, centered at t
dyt = dy[::2,1::2] + dy[1::2,1::2]

# x-distance between  q-points, centered at v
dxCv = dx[2::2,::2] + dx[2::2,1::2]

# y-distance between  q-points, centered at u
dyCu = dy[::2,2::2] + dy[1::2,2::2]

# x-distance between t-points, centered at u
dxCu = dx[1::2,1::2] + np.roll(dx[1::2,1::2], -1, axis=-1)

# y-distance between t-points, centered at v
dyCv = dy[1::2,1::2] + np.roll(dy[1::2,1::2], -1, axis=0)


Define a function to write variables to the netcdf file to be created:

In [7]:
def write_nc_var(var, name, dimensions, long_name=None, units=None, coordinates=None):
    nc.createVariable(name, 'f8', dimensions)
    if long_name is not None:
        nc.variables[name].long_name = long_name
    if units is not None:
        nc.variables[name].units = units
    if coordinates is not None:
        nc.variables[name].coordinates = coordinates
    nc.variables[name][:] = var
    print ' ... wrote variable', name


Write the netcdf file:

In [8]:
filename = 'tx0.66v1_grid.nc'
nc = netCDF4.Dataset(filename, 'w', format='NETCDF3_64BIT')
nc.Description = 'MOM6 2/3 degree tripolar grid (ORCA type) with tropical stretching'
# write the author, e.g.,:
# nc.Author = 'Fred Castruccio (fredc@ucar.edu)'
nc.Created = datetime.now().isoformat()
nc.type = 'tx0.66v1 file'

# grid aspect ratio                    
ar = dyt / dxt

# grid effective grid spacing
# A = 4*pi*r^2 , area of sphere of radius r
# dA = (r*cos(theta)*dlambda)*(r*dtheta), differential area on sphere                                              
#    = r^2*domega                                                                                                  
# domega = dA/r^2, differential solid angle  (steradians, sr)                                                      
# 1 sr = (180./pi)^2 square degrees                                                                                
costheta = np.cos(tlat*np.pi/180.)
rearth = 637122000 # Earth radius in centimeter                                                                    
domega = tarea / rearth**2
egs  = np.sqrt(domega * (180./np.pi)**2)

# create netcdf dimension
M, L = qlon.shape
nc.createDimension('nyp', M)
nc.createDimension('nxp', L)
nc.createDimension('ny', M-1)
nc.createDimension('nx', L-1)


# write variable
write_nc_var(tlon, 'tlon', ('ny', 'nx'), 'array of t-grid longitudes', 'degrees_east')
write_nc_var(tlat, 'tlat', ('ny', 'nx'), 'array of t-grid latitudes', 'degrees_north')
write_nc_var(ulon, 'ulon', ('ny', 'nxp'), 'array of u-grid longitudes', 'degrees_east')
write_nc_var(ulat, 'ulat', ('ny', 'nxp'), 'array of u-grid latitudes', 'degrees_north')
write_nc_var(vlon, 'vlon', ('nyp', 'nx'), 'array of v-grid longitudes', 'degrees_east')
write_nc_var(vlat, 'vlat', ('nyp', 'nx'), 'array of v-grid latitudes', 'degrees_north')
write_nc_var(qlon, 'qlon', ('nyp', 'nxp'), 'array of q-grid longitudes', 'degrees_east')
write_nc_var(qlat, 'qlat', ('nyp', 'nxp'), 'array of q-grid latitudes', 'degrees_north')

write_nc_var(dxt, 'dxt', ('ny', 'nx'), 'x-distance between u-points, centered at t', 'meters')
write_nc_var(dyt, 'dyt', ('ny', 'nx'), 'y-distance between v-points, centered at t', 'meters')
write_nc_var(dxCv, 'dxCv', ('ny', 'nx'), 'x-distance between  q-points, centered at v', 'meters')
write_nc_var(dyCu, 'dyCu', ('ny', 'nx'), 'y-distance between  q-points, centered at u', 'meters')
write_nc_var(dxCu, 'dxCu', ('ny', 'nx'), 'x-distance between  t-points, centered at u', 'meters')
write_nc_var(dyCv, 'dyCv', ('ny', 'nx'), 'y-distance between  t-points, centered at v', 'meters')
write_nc_var(tarea, 'tarea', ('ny', 'nx'), 'area of t-cells', 'meters^2')
write_nc_var(tmask, 'tmask', ('ny', 'nx'), 'ocean fraction at t-cell centers', 'none')
write_nc_var(angle, 'angle', ('ny', 'nx'), 'angle grid makes with latitude line', 'degrees')
write_nc_var(depth, 'depth', ('ny', 'nx'), 'depth at h points', 'meters')
write_nc_var(ar, 'ar', ('ny', 'nx'), 'grid aspect ratio (dyt/dxt)', 'none')
write_nc_var(egs, 'egs', ('ny', 'nx'), 'grid effective grid spacing', 'degrees')

# close files
nc.close()






 ... wrote variable tlon
 ... wrote variable tlat
 ... wrote variable ulon
 ... wrote variable ulat
 ... wrote variable vlon
 ... wrote variable vlat
 ... wrote variable qlon
 ... wrote variable qlat
 ... wrote variable dxt
 ... wrote variable dyt
 ... wrote variable dxCv
 ... wrote variable dyCu
 ... wrote variable dxCu
 ... wrote variable dyCv
 ... wrote variable tarea
 ... wrote variable tmask
 ... wrote variable angle
 ... wrote variable depth
 ... wrote variable ar
 ... wrote variable egs
